In [1]:
import numpy as np
import torch
from scipy import stats
from lib.DataManager import *
from lib.PolicyStats import *
import os
import cma
from cma.constraints_handler import AugmentedLagrangian, PopulationEvaluator
from IPython import display
import pickle

In [7]:
def policy_softmax(policy):
    numerators = np.exp(policy)
    return (numerators.T / np.sum(numerators, axis=1)).T

In [2]:
num_policies = 100
es = pickle.load(open("pickles//saved-cma-gw-is-20.pkl", "rb"))
print("ES_Convergence : " + str(sum(es.mean**2)))

ES_Convergence : 85.04553640730798


In [5]:
greed_policy = np.argmax(es.result.xbest.reshape(23,4),axis=1)
print(np.insert(greed_policy, [12,16],[-1,-1]).reshape((5,5)))

[[ 1  1  3  2  2]
 [ 1  0  1  2  1]
 [ 0  0 -1  3  2]
 [ 1  2 -1  3  3]
 [ 0  2  3  2  1]]


In [8]:
policy_softmax(es.result.xbest.reshape(23,4))

array([[0.10949767, 0.60798263, 0.17897431, 0.10354539],
       [0.0643022 , 0.47355261, 0.3133417 , 0.14880349],
       [0.32165237, 0.20080725, 0.0369522 , 0.44058818],
       [0.21228228, 0.08041023, 0.61956999, 0.08773749],
       [0.0698699 , 0.20836866, 0.59440639, 0.12735505],
       [0.03609926, 0.6519258 , 0.15292854, 0.1590464 ],
       [0.6375451 , 0.1865995 , 0.16037985, 0.01547555],
       [0.04826809, 0.83663126, 0.07352048, 0.04158017],
       [0.02497457, 0.30065161, 0.60525945, 0.06911437],
       [0.23542035, 0.30270864, 0.18192164, 0.27994937],
       [0.41940101, 0.32796263, 0.11342856, 0.13920779],
       [0.51603337, 0.08554994, 0.18054402, 0.21787267],
       [0.23905087, 0.01296784, 0.18179429, 0.566187  ],
       [0.10593255, 0.0909279 , 0.73574312, 0.06739643],
       [0.22550855, 0.45418854, 0.22200703, 0.09829587],
       [0.35362845, 0.10218828, 0.37079136, 0.17339192],
       [0.07504114, 0.17110136, 0.34766438, 0.40619312],
       [0.39458078, 0.13952392,